In [29]:
import pandas as pd
import numpy as np
df = pd.read_csv('nfa_2019_public_data.csv')

In [30]:
df.head()

,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,qscore
0,Armenia,1992,1,AreaPerCap,0.140292,0.199546,0.097188051,0.036888,0.029320,0.000000,5.032351e-01,3A
1,Armenia,1992,1,AreaTotHA,483000.000000,687000.000000,334600,127000.000000,100943.000800,0.000000,1.732543e+06,3A
2,Armenia,1992,1,BiocapPerCap,0.159804,0.135261,0.084003213,0.013742,0.033398,0.000000,4.262086e-01,3A
3,Armenia,1992,1,BiocapTotGHA,550176.242700,465677.972200,289207.1078,47311.551720,114982.279300,0.000000,1.467355e+06,3A
4,Armenia,1992,1,EFConsPerCap,0.387510,0.189462,1.26E-06,0.004165,0.033398,1.114093,1.728629e+00,3A


In [31]:
df[ 'qscore' ].value_counts()
df.isna().sum()
#for simplicity, we will drop the rows with missing values.

country               0
year                  0
country_code          0
record                0
crop_land         20472
grazing_land      20472
forest_land       20472
fishing_ground    20473
built_up_land     20473
carbon            20473
total                 9
qscore                1
dtype: int64

In [32]:
df = df.dropna()
df.isna().sum()

country           0
year              0
country_code      0
record            0
crop_land         0
grazing_land      0
forest_land       0
fishing_ground    0
built_up_land     0
carbon            0
total             0
qscore            0
dtype: int64

An obvious change in our target variable after removing the missing values is that there are only three classes left and from 
# the distribution of the 3 classes, we can see that there is an obvious imbalance between the classes. 
# There are methods that can be applied to handle this imbalance such as oversampling and undersampling.
# Oversampling involves increasing the number of instances in the class with fewer instances, while undersampling involves reducing the data points in the class with more instances.

# For now, we will convert this to a binary classification problem by combining class '2A'and '1A'. 

In [33]:
df[ 'qscore' ] = df[ 'qscore' ].replace([ '1A' ], '2A' )
df.qscore.value_counts()

3A    51473
2A      240
Name: qscore, dtype: int64

In [34]:
df_2A = df[df.qscore== '2A' ]
df_3A = df[df.qscore== '3A' ].sample( 350 )
data_df = df_2A.append(df_3A)

In [36]:
import sklearn.utils
data_df = sklearn.utils.shuffle(data_df)
data_df = data_df.reset_index(drop= True )
data_df.shape
data_df.qscore.value_counts()

3A    350
2A    240
Name: qscore, dtype: int64

In [ ]:
# more preprocessing
data_df = data_df.drop(columns=[ 'country_code' , 'country' , 'year' ])

X = data_df.drop(columns= 'qscore' )
y = data_df[ 'qscore' ]

In [39]:
# split the data into training and testing sets
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= 0.3 , random_state= 0 )
y_train.value_counts()

3A    244
2A    169
Name: qscore, dtype: int64

In [42]:
#There is still an imbalance in the class distribution. For this, we use SMOTE only on the training data to handle this

# encode categorical variable
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
x_train.record = encoder.fit_transform(x_train.record)
x_test.record = encoder.transform(x_test.record)

In [45]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state= 1 )
x_train_balanced, y_balanced = smote.fit_sample(x_train, y_train)

In [48]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_train_df = scaler.fit_transform(x_train_balanced.drop(columns=[ 'record' ]))
normalised_train_df = pd.DataFrame(normalised_train_df,
                                   columns=x_train_balanced.drop(columns=[ 'record' ]).columns)
normalised_train_df[ 'record' ] = x_train_balanced[ 'record' ]

In [49]:
x_test = x_test.reset_index(drop= True )
normalised_test_df = scaler.transform(x_test.drop(columns=[ 'record' ]))
normalised_test_df = pd.DataFrame(normalised_test_df,
                                  columns=x_test.drop(columns=[ 'record' ]).columns)
normalised_test_df[ 'record' ] = x_test[ 'record' ] 

In [52]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(normalised_train_df, y_balanced)

LogisticRegression()

In [53]:
# returns
LogisticRegression()

LogisticRegression()

# Confusion Matrix

In [55]:
from sklearn.metrics import recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
new_predictions = log_reg.predict(normalised_test_df)
cnf_mat = confusion_matrix(y_true=y_test, y_pred=new_predictions, labels=[ '2A', '3A' ])
cnf_mat

array([[47, 24],
       [54, 52]], dtype=int64)

# Measuring Classification Performance Cross-validation and accuracy

In [57]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(log_reg, normalised_train_df, y_balanced, cv= 5 , scoring= 'f1_macro' ) 
scores

array([0.48787625, 0.4897605 , 0.46582809, 0.5245098 , 0.48941884])

# Acurracy

In [58]:
accuracy = accuracy_score(y_true=y_test, y_pred=new_predictions)
print( 'Accuracy: {}' .format(round(accuracy* 100 ), 2 ))

Accuracy: 56


# Precision

In [59]:
precision = precision_score(y_true=y_test, y_pred=new_predictions, pos_label= '2A' )
print( 'Precision: {}' .format(round(precision* 100 ), 2 ))

Precision: 47


# Recall

In [60]:
recall = recall_score(y_true=y_test, y_pred=new_predictions, pos_label= '2A' )
print( 'Recall: {}' .format(round(recall* 100 ), 2 ))

Recall: 66


# F1-Score

In [61]:
f1 = f1_score(y_true=y_test, y_pred=new_predictions, pos_label= '2A' )
print( 'F1: {}' .format(round(f1* 100 ), 2 ))

F1: 55


# K-Fold Cross Validation

In [66]:
from sklearn.model_selection import KFold
kf = KFold(n_splits= 5 )
kf.split(normalised_train_df)
f1_scores = []
# run for every split
for train_index, test_index in kf.split(normalised_train_df):
 x_train, x_test = normalised_train_df.iloc[train_index], normalised_train_df.iloc[test_index]
 y_train, y_test = y_balanced[train_index], y_balanced[test_index]
 model = LogisticRegression().fit(x_train, y_train)
 # save result to list
f1_scores.append(f1_score(y_true=y_test, y_pred= model.predict(x_test), pos_label= '2A' )* 100) 

# Stratified K-Fold Cross Validation

In [70]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits= 5 , shuffle= True , random_state= 1 )
f1_scores = []
# Run for every split
for train_index, test_index in skf.split(normalised_train_df, y_balanced):
    x_train, x_test = np.array(normalised_train_df)[train_index], np.array(normalised_train_df)[test_index]
    y_train, y_test = y_balanced[train_index], y_balanced[test_index]
    model = LogisticRegression().fit(x_train, y_train)
#save result to list
f1_scores.append(f1_score(y_true=y_test, y_pred=model.predict(x_test), pos_label= '2A' ))

# Leave One Out Cross Validation (LOOCV)

In [74]:
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
scores = cross_val_score(LogisticRegression(), normalised_train_df, y_balanced, cv=loo,
 scoring= 'f1_macro' )
average_score = scores.mean() * 100